In [2]:
model_path = 's3://sagemaker-us-east-2-220544705075/Flower-classification-2023-10-06-09-24-54-222/output/model.tar.gz'

In [3]:
from sagemaker.tensorflow.serving import Model

role = "arn:aws:iam::220544705075:role/agriculture"

tf_model = Model(model_path,
           role = role,
           framework_version='2.3')

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\TEXON WARE\AppData\Local\sagemaker\sagemaker\config.yaml


The class sagemaker.tensorflow.serving.Model has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\TEXON WARE\AppData\Local\sagemaker\sagemaker\config.yaml


In [4]:
type(tf_model)

sagemaker.deprecations.deprecated_class.<locals>.DeprecatedClass

In [9]:
import time

%%time
tf_endpoint_name = 'flower-'+time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

tf_predictor = tf_model.deploy(initial_instance_count=1,
                                   instance_type='ml.t2.medium',        # $0.115/hour 
                                   endpoint_name=tf_endpoint_name) 

'%time' is not recognized as an internal or external command,
operable program or batch file.


sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\TEXON WARE\AppData\Local\sagemaker\sagemaker\config.yaml
-----!

### Prediction

In [10]:
type(tf_predictor)

sagemaker.tensorflow.model.TensorFlowPredictor

In [11]:
from keras.preprocessing import image
import os
import requests
from io import BytesIO
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

# Download an image and save it as test.jpg
url='https://images.all-free-download.com/images/graphiclarge/background_bloom_blossom_celebration_ceremony_603818.jpg'

response = requests.get(url)
img1 = Image.open(BytesIO(response.content))
img1.save('./test.jpg')
# print(os.listdir('/tmp/'))
    
file_name = 'test.jpg'
print(file_name)

In [14]:
# image preprocessing
test_image = Image.open(file_name)
test_image = test_image.resize((300, 300))
test_image = np.asarray(test_image)/255.0
test_image=np.expand_dims(test_image, axis=0)

In [17]:
test_image.shape
all_labels = {0: 'daisy', 1: 'dandelion', 2: 'rose', 3: 'sunflower', 4: 'tulip'}

In [24]:
result =tf_predictor.predict(test_image)
# print(result['predictions'][0])

prediction = np.array(result['predictions'])
# print("result:",prediction)
mp = np.max(prediction[0], axis=-1)
    
labels = all_labels
    
print("\n\nMaximum Probability: ", mp)
predicted_class = labels[np.argmax(prediction[0], axis=-1)]
print("Classified:", predicted_class, "\n\n")
    
classes=[]
prob=[]
print("\n-------------------Individual Probability--------------------------------\n")

for i,j in enumerate (prediction[0],0):
    print(labels[i].upper(),':',round(j*100,2),'%')
    classes.append(labels[i])
    prob.append(round(j*100,2))



Maximum Probability:  0.99966383
Classified: rose 



-------------------Individual Probability--------------------------------

DAISY : 0.0 %
DANDELION : 0.0 %
ROSE : 99.97 %
SUNFLOWER : 0.0 %
TULIP : 0.03 %


### Delete Endpoint

In [25]:
tf_predictor.delete_endpoint()